# Train Football Player Detector

### Before you start
Use `nvidia-smi` command to see your GPU specs (only if you have a nvidia GPU, it doesn't work on my macbook)

This is mostly useful to check if you are working with a GPU environment on Google Colab, if
you receive the msg: 'command not found: nvidia-smi' then you need to change the environment

In [1]:
!nvidia-smi

zsh:1: command not found: nvidia-smi


Make sure which directory you are working on (useful when working in remote)

In [2]:
import os
HOME = os.getcwd()
print(HOME)

/Users/alex/Desktop/IA/Football_Analysis/notebooks


### Install dependencies

In [3]:
%pip install -q ultralytics roboflow

Note: you may need to restart the kernel to use updated packages.


### Imports

In [ ]:
from roboflow import Roboflow
from dotenv import find_dotenv, load_dotenv
from google.colab import userdata
from IPython.display import Image

### Configure API keys

run this one only locally

In [ ]:
# find .env automatically by walking up directories until it's found
dotenv_path = find_dotenv()

# load up the entries as environment variables
load_dotenv(dotenv_path)

ROBOFLOW_API_KEY = os.getenv("ROBOFLOW_API_KEY")

True

run this one only on Google Colab

In [ ]:
ROBOFLOW_API_KEY = userdata.get('ROBOFLOW_API_KEY')

### Pull dataset from roboflow

run this one only locally

In [7]:
%cd ..
!mkdir datasets
%cd datasets

/Users/alex/Desktop/IA/Football_Analysis
mkdir: datasets: File exists
/Users/alex/Desktop/IA/Football_Analysis/datasets


run this one only on google colab

In [ ]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

This dataset is already prepared for model training, we use yolov5 because is the one that
works better recognizing the ball

In [ ]:
rf = Roboflow(api_key=ROBOFLOW_API_KEY)

project = rf.workspace("roboflow-jvuqo").project("football-players-detection-3zvbc")
version = project.version(1)
dataset = version.download("yolov5")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to football-players-detection-1 in yolov5pytorch:: 100%|██████████| 1338/1338 [00:00<00:00, 2191.53it/s]


Update `data.yaml` structure to align with the latest requirements of the ultralytics package

In [10]:
import shutil

shutil.move('football-players-detection-1/train',
            'football-players-detection-1/football-players-detection-1/train')

shutil.move('football-players-detection-1/test',
            'football-players-detection-1/football-players-detection-1/test')

shutil.move('football-players-detection-1/valid',
            'football-players-detection-1/football-players-detection-1/valid')

'football-players-detection-1/football-players-detection-1/valid'

### Custom Training

Using big image size to detect the ball consistently (`imgsz=1280`, `epochs=50`, `batch=6`). 24 GiB of GPU memory is needed to run this training. We can download a pre-trained model if we don't have access to this hardware.

But to try the process of training a model we can set a smaller model and image size. (I only runned this code on Google Colab)

In [ ]:
%cd {HOME}

!yolo task=detect mode=train model=yolov5x.pt data={dataset.location}/data.yaml epochs=100 imgsz=640

### Validate custom model
Benchmark the model to verify everything went well, we will use the maP metric to measure the average precission of our model.

In [ ]:
%cd {HOME}

!yolo task=detect mode=val model={HOME}/runs/detect/train/weights/best.pt data={dataset.location}/data.yaml imgsz=1280